In [23]:
from bs4 import BeautifulSoup
import json
import csv
import time, datetime
import requests

In [24]:
def get_datetime():
    return datetime.datetime.utcfromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S')

In [25]:
# Mens Tshirt URL
url = "http://www.jabong.com/men/clothing/polos-tshirts/?sort=popularity&dir=desc&limit=2000&page="
# Open csv file for writing
info_csv = open('info_' + get_datetime() + '.csv', 'w')
info_writer = csv.writer(info_csv, delimiter=',')
# Write headers
info_writer.writerow(['ID', 'Title', 'Original Price', 'Discount Price', 'Product URL', 'Product Image URL'])
info_writer.writerow([])

In [27]:
items = 0                   # Items count
page = 1                    # Page count
proc_time = 0               # Processing time
img_urls = []               # Image url list
start_time = time.time()    # Start time
LIMIT = 10000               # Number of shirts to fetch
while items < LIMIT:
    # Get contents of Page #page
    content = requests.get(url + str(page))
    proc_start_time = time.time()
    # Cook soup of contents
    soup = BeautifulSoup(content.text, 'lxml')
    # Get all products
    shirts = soup.find_all('div', class_='product-tile')
    for shirt in shirts:
        # Skip dummy product
        if shirt.a['href'] == u'#':
            continue
        sid = shirt['data-product-id']
        title = shirt.find('div', class_='product-info').div.text
        prices = shirt.find_all('span', class_='standard-price')
        original_price = prices[0].string
        # If there is no discount
        discount_price = original_price
        # Else if there is a discount
        if len(prices) > 1:
            discount_price = shirt.find_all('span', class_='standard-price')[1].string
        surl = shirt.a['href']
        js = json.loads(shirt.img['data-img-config'])
        img_url = js['base_path'] + js['500']
        img_urls.append(img_url)
        # Write to csv
        row = [sid, title, original_price, discount_price, surl, img_url]
        info_writer.writerow([ unicode(s).encode("utf-8") for s in row ])
        info_csv.flush()
        items += 1
    proc_time += time.time() - proc_start_time
    print 'Wrote ' + str(items) + ' items in ' + str(datetime.timedelta(seconds=(time.time() - proc_start_time)))
    page += 1
exec_time = time.time() - start_time
exec_time = str(datetime.timedelta(seconds=exec_time))
print "\nTotal Execution time : "  + exec_time
proc_time = str(datetime.timedelta(seconds=proc_time))
print "Processing time : "  + proc_time
# Close csv file
info_csv.close()

Got 100 items in 0:00:00.404302
Got 198 items in 0:00:00.352211
Got 298 items in 0:00:00.347644
Got 398 items in 0:00:00.411996
Got 497 items in 0:00:00.349374
Got 596 items in 0:00:00.424933
Got 694 items in 0:00:00.354724
Got 794 items in 0:00:00.345816
Got 894 items in 0:00:00.427170
Got 994 items in 0:00:00.360525
Got 1094 items in 0:00:00.405840
Got 1194 items in 0:00:00.366124
Got 1294 items in 0:00:00.355564
Got 1393 items in 0:00:00.411554
Got 1493 items in 0:00:00.349396
Got 1593 items in 0:00:00.420140
Got 1693 items in 0:00:00.353533
Got 1793 items in 0:00:00.344726
Got 1892 items in 0:00:00.413604
Got 1991 items in 0:00:00.356253
Got 2090 items in 0:00:00.361162
Got 2190 items in 0:00:00.449522
Got 2290 items in 0:00:00.354803
Got 2389 items in 0:00:00.423481
Got 2488 items in 0:00:00.353197
Got 2588 items in 0:00:00.376669
Got 2688 items in 0:00:00.453888
Got 2787 items in 0:00:00.351019
Got 2887 items in 0:00:00.405220
Got 2987 items in 0:00:00.357206
Got 3086 items in 0:

In [29]:
def get_images(imgs):
    for img in imgs:
        subprocess.call('wget -q -P images "' + img + '"',  shell=True)
start_time = time.time()
get_images(img_urls)
fetch_time = time.time() - start_time
fetch_time = str(datetime.timedelta(seconds=fetch_time))
print "Images fetched in " + fetch_time